# Main Coding Phase.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [2]:
data = pd.read_csv('final_data.csv')
data.head(2)

,Unnamed: 0.1,Unnamed: 0,title,text,label,real_fake_news
0,0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,0


In [3]:
del data['Unnamed: 0.1']

In [4]:
data.head()

,Unnamed: 0,title,text,label,real_fake_news
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


In [5]:
data = data.drop(['Unnamed: 0' , 'label'] , axis = 1)

<h3><u>Final DataFram Ready.</u><h3>

In [6]:
data.head()

,title,text,real_fake_news
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


<h3><u>Shuffle Data</u></h3>

In [7]:
data = data.sample(frac=1)

In [8]:
data.head(3)

,title,text,real_fake_news
2279,17 biblical rules for marriage the Kim Davis s...,Some people believe that Kentucky—or even all ...,1
2161,Devastating Wiki Leaks Show Scope Of Clinton C...,"Leave a reply \nOn the October 27, 2016 Fox Ne...",0
1761,"“Wikileaks is the Mossad, Stupid, Not the Russ...",Russian experts collecting evidence of anti-go...,0


<h3><u>Check The Data Set Having any null value or not ? </u></h3>

In [9]:
data.isnull().sum() # check the  null value 

title             0
text              0
real_fake_news    0
dtype: int64

<h3><u>for remove any special character in the given string for that <span>(RemoveData)<span> define one function </u></h3> 

In [10]:
def RemoveData(text):
    text  = text.lower()
    text = re.sub('\[.*?\]','' , text)
    text = re.sub("\\W"," ",text)
    text = re.sub('http?://\S+|www.\.\S+' ,'',text)
    text = re.sub('<.*?>+', '',text)
    text = re.sub('[&%s]' % re.escape(string.punctuation) , '' , text)
    text = re.sub('\n' , '' , text)
    text = re.sub('\w*\d\w*' , '' , text )
    return text

In [11]:
data['text'] = data['text'].apply(RemoveData)

### data.head(3)

<h3><u>dependent and independent variable define </u></h3>

In [12]:
x= data['text']
y= data['real_fake_news']

In [13]:
x_train , x_test , y_train , y_test  = train_test_split(x,y , test_size = .25)

In [14]:
from sklearn.feature_extraction.text import *

In [15]:
vect = TfidfVectorizer()

In [16]:
xv_train = vect.fit_transform(x_train)
xv_test  = vect.transform(x_test)

<h3><u>Use logistic regresssion in to the our  code </u></h3>

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
lr = LogisticRegression()
lr.fit(xv_train ,y_train)

LogisticRegression()

In [19]:
lr.score(xv_test , y_test)

0.9210858585858586

In [20]:
pred_lr = lr.predict(xv_test)

In [21]:
print(classification_report(y_test , pred_lr))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92       806
           1       0.93      0.91      0.92       778

    accuracy                           0.92      1584
   macro avg       0.92      0.92      0.92      1584
weighted avg       0.92      0.92      0.92      1584



<h3><u>Random Forest Classifier </u></h3>

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
rfc =RandomForestClassifier(random_state=0)

In [24]:
rfc.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [25]:
rfc.score(xv_test, y_test)

0.9015151515151515

In [26]:
pred_rfc = rfc.predict(xv_test)

In [27]:
print(classification_report(y_test , pred_rfc))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       806
           1       0.90      0.90      0.90       778

    accuracy                           0.90      1584
   macro avg       0.90      0.90      0.90      1584
weighted avg       0.90      0.90      0.90      1584



# Manual Testing 

In [28]:
def output_lable(n):
    if n == 0 :
        return "Fake News"
    elif n == 1 :
        return "Not A fake News"
def manual_testing(news):
    testing_news = {"test":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test['test'].apply(RemoveData)
    new_x_test = new_def_test['test']
    new_xv_test = vect.transform(new_x_test)
    pred_lr = lr.predict(new_xv_test)
    pred_rfc = rfc.predict(new_xv_test)
    
    return print(f"LR : {output_lable(pred_lr)},\nRFC : {output_lable(pred_rfc)} \n" )
    
    

In [30]:
news = str(input("enter news : --> "))
print(news)
manual_testing(news)

enter news : -->  helloo oooooo h eoiufb


helloo oooooo h eoiufb
LR : Fake News,
RFC : Fake News 

